# Определение активности для дня недели, недели года, дня месяца на курсе

Импортируем библиотеки и делаем некоторые настройки

In [1]:
import pandas as pd
import numpy as np

In [2]:
#sns.set(rc={'figure.figsize': (20, 10)})

In [3]:
import datetime, calendar

In [4]:
import warnings
warnings.filterwarnings('ignore')

Загружаем данные о действиях и их статусе. Объединяем все в одну таблицу

In [5]:
events = pd.read_csv('DATA\\events_all_two_days.zip', compression='zip')

In [6]:
submissions = pd.read_csv('DATA\\submissions_all_two_days.zip', compression='zip')

In [7]:
events = events.rename(columns={'action': 'action_submission_status'})
submissions = submissions.rename(columns={'submission_status': 'action_submission_status'})

In [8]:
data = pd.concat([events, submissions])

In [9]:
del events
del submissions

In [10]:
data.sort_values(by=['user_id', 'timestamp'], axis=0, inplace=True)

Для сбалансированности отберем данные только за три полных года 2016, 2017, 2018

In [11]:
d_1 = datetime.datetime(2016, 1, 1, 0, 0, 0)
d_2 = datetime.datetime(2019, 1, 1, 0, 0, 0)
first_timestamp = calendar.timegm(d_1.timetuple())
second_timestamp = calendar.timegm(d_2.timetuple())

In [12]:
data = data[(data.timestamp >= first_timestamp) & \
            (data.timestamp < second_timestamp)]

In [13]:
print(data.date.min(),'//', data.date.max())

2016-01-01 // 2018-12-31


Определим активности пользователей на курсе

koef_weekday - активность для дня недели,
koef_week    - активность номера недели,
koef_day     - активность дня месяца

In [14]:
# коэффициенты-признаки
koef_weekday = (data.groupby('weekday').user_id.nunique()/ \
                data.groupby('weekday').user_id.nunique().sum())\
                .to_frame().reset_index() \
                .rename(columns={'user_id': 'koef_wd'})
koef_week = (data.groupby('week').user_id.nunique()/ \
             data.groupby('week').user_id.nunique().sum())\
             .to_frame().reset_index() \
             .rename(columns={'user_id': 'koef_wk'})
koef_day = (data.groupby('day').user_id.nunique()/ \
            data.groupby('day').user_id.nunique().sum())\
            .to_frame().reset_index() \
            .rename(columns={'user_id': 'koef_dm'})
# поправки из-за непредставительности некоторых данных
koef_week.koef_wk[51] = (koef_week.koef_wk[51])*21/20
koef_week.koef_wk[52] = (koef_week.koef_wk[52])*21/3
koef_day.koef_dm[28] = (koef_day.koef_dm[28])*36/34
koef_day.koef_dm[29] = (koef_day.koef_dm[29])*36/33
koef_day.koef_dm[30] = (koef_day.koef_dm[30])*36/21
#переномируем некоторые после изменений
koef_week.koef_wk = koef_week.koef_wk / koef_week.koef_wk.sum()
koef_day.koef_dm = koef_day.koef_dm / koef_day.koef_dm.sum()

Сохраним результаты в файл:

In [15]:
koef_weekday.to_csv('DATA\\koef_weekday.csv', index=False)
koef_week.to_csv('DATA\\koef_week.csv', index=False)
koef_day.to_csv('DATA\\koef_day.csv', index=False)